<a href="https://colab.research.google.com/github/AmanBiswas/cancer_detection/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from PIL import Image
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from tensorflow.keras.preprocessing import image

In [8]:
csv_path="/content/drive/MyDrive/Colab Notebooks/cancer cell detection/data/HAM10000_metadata.csv"
image_folder = '/content/drive/MyDrive/Colab Notebooks/cancer cell detection/data/HAM10000_images_part_1'
image_folder2 = '/content/drive/MyDrive/Colab Notebooks/cancer cell detection/data/HAM10000_images_part_2'

In [9]:
csv_data=pd.read_csv(csv_path)

In [10]:
class_to_label = {name: index for index, name in enumerate(csv_data['dx'].unique())}

In [11]:
csv_data.head

<bound method NDFrame.head of          lesion_id      image_id     dx dx_type   age     sex localization
0      HAM_0000118  ISIC_0027419    bkl   histo  80.0    male        scalp
1      HAM_0000118  ISIC_0025030    bkl   histo  80.0    male        scalp
2      HAM_0002730  ISIC_0026769    bkl   histo  80.0    male        scalp
3      HAM_0002730  ISIC_0025661    bkl   histo  80.0    male        scalp
4      HAM_0001466  ISIC_0031633    bkl   histo  75.0    male          ear
...            ...           ...    ...     ...   ...     ...          ...
10010  HAM_0002867  ISIC_0033084  akiec   histo  40.0    male      abdomen
10011  HAM_0002867  ISIC_0033550  akiec   histo  40.0    male      abdomen
10012  HAM_0002867  ISIC_0033536  akiec   histo  40.0    male      abdomen
10013  HAM_0000239  ISIC_0032854  akiec   histo  80.0    male         face
10014  HAM_0003521  ISIC_0032258    mel   histo  70.0  female         back

[10015 rows x 7 columns]>

In [12]:
images = []
labels = []

for index, row in csv_data.iterrows():
    image_path = f"{image_folder}/{row['image_id']}.jpg"
    if not os.path.exists(image_path):
        image_path = f"{image_folder2}/{row['image_id']}.jpg"

    image = load_img(image_path, target_size=(100, 100))
    image = np.array(image)
    label = class_to_label[row['dx']]

    images.append(image)
    labels.append(label)

In [13]:
images=images[:10000]
labels=labels[:10000]
# Convert the lists into numpy arrays
images = np.array(images)
labels = np.array(labels)

In [14]:
images = images / 255.0


In [15]:
num_classes = len(class_to_label)

In [16]:
import random
import numpy as np

# Assuming you have loaded your dataset into X_train and y_train

# Combine X_train and y_train
dataset = list(zip(images,labels))

# Shuffle the dataset
random.shuffle(dataset)

# Unpack the shuffled dataset into X_train and y_train
images,labels = zip(*dataset)

In [17]:
# Convert X_train and y_train back to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [18]:
from sklearn.model_selection import train_test_split
# x_train,x_test=images[:9000],images[9000:]
# y_train,y_test=labels[:9000],labels[9000:]
x_train,x_test,y_train,y_test=train_test_split(images,labels,test_size=0.2)
# del images
# del labels

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [20]:
target_size = (100, 100)
num_classes = 7
img_height = 100
img_width=100
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [22]:
epochs=10
model.fit(
  x=x_train,y= y_train,
    batch_size=100,
  epochs=epochs
)

Epoch 1/10
80/80 [==============================] - 187s 2s/step - loss: 1.0614 - accuracy: 0.6620
Epoch 2/10
80/80 [==============================] - 183s 2s/step - loss: 0.9211 - accuracy: 0.6762
Epoch 3/10
80/80 [==============================] - 183s 2s/step - loss: 0.8671 - accuracy: 0.6875
Epoch 4/10
80/80 [==============================] - 186s 2s/step - loss: 0.8109 - accuracy: 0.6969
Epoch 5/10
80/80 [==============================] - 187s 2s/step - loss: 0.7581 - accuracy: 0.7185
Epoch 6/10
80/80 [==============================] - 184s 2s/step - loss: 0.7092 - accuracy: 0.7352
Epoch 7/10
80/80 [==============================] - 181s 2s/step - loss: 0.6817 - accuracy: 0.7462
Epoch 8/10
80/80 [==============================] - 184s 2s/step - loss: 0.6534 - accuracy: 0.7552
Epoch 9/10
80/80 [==============================] - 186s 2s/step - loss: 0.6221 - accuracy: 0.7651
Epoch 10/10
80/80 [==============================] - 179s 2s/step - loss: 0.6000 - accuracy: 0.7785


In [23]:
loss,accuracy=model.evaluate(x_test,y_test)
accuracy

63/63 [==============================] - 14s 226ms/step - loss: 0.7246 - accuracy: 0.7415


0.7415000200271606

In [24]:
model.save('skin_cancer_model.h5')

In [25]:
new_model = keras.models.load_model('skin_cancer_model.h5')

In [26]:
loss, acc = new_model.evaluate(x_train, y_train, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

250/250 - 50s - loss: 0.5714 - accuracy: 0.7800 - 50s/epoch - 200ms/step
Restored model, accuracy: 78.00%
